# Perceptron Notebook!

## 1.  Perceptron / McCulloch-Pitts Neuron

$$\hat{y}(x) = H(\sum_{i=1}^{n}w_ix_i)$$

or

$$\hat{y}(x) = H(w \cdot x)$$

or

$$\hat{y}(x) = H(w^Tx)$$

where $H$ is the heaviside step function

In [ ]:
import random
import torch
import torch.nn as nn
from torch import Tensor
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class Perceptron():
    def __init__(self, w, b):
        self.w = w
        self.b = b

    def predict(self, x: Tensor):
        
        ################################
        # Fill Out Prediction Function #
        ################################

## 2. Let's Test Our Answers

In [ ]:
# create an instance of the perceptron with weights (1, -1) and bias 0
q1 = Perceptron(torch.tensor([1.0, -1.0]), 0)
x = torch.tensor([0.5, 0.8])
y_hat = q1.predict(x)
y_hat

In [ ]:
# instance of the perceptron with weights (0.2, 0.75, -0.5) and bias -3
q2 = Perceptron(torch.tensor([0.2, 0.75, -0.5]), -3)
x = torch.tensor([10.0, 20.0, 30.0])
y_hat = q2.predict(x)
y_hat

In [ ]:
# instance of the perceptron with weights (-2, -2) and bias 3
q3456 = Perceptron(torch.tensor([-2, -2]), 3)

x_hist = []
y_hats = []
for i in range(2):
    for j in range(2):
        x = torch.tensor([i, j])
        x_hist.append(x)
        y_hats.append(q3456.predict(x))

print(x_hist)
print(y_hats)

## 3. Perceptron Learning Algorithm

#### 3.1 Plotting Function

In [ ]:
def plot_perceptron(neuron, max_x, min_x, x_coords_0, y_coords_0, x_coords_1, y_coords_1):
        # Create the plot
        plt.scatter(x_coords_0, y_coords_0, c='red', label='Class 0')
        plt.scatter(x_coords_1, y_coords_1, c='blue', label='Class 1')

        # Add titles and labels
        plt.title('Points')
        plt.xlabel('X-axis')
        plt.ylabel('Y-axis')
        plt.grid(True)  # Optional: add a grid

        # Plot Weight Vector
        plt.arrow(0,0,neuron.w[0], neuron.w[1], width=0.02, head_width=0.05)

        #Calculate the boundary line variables
        if neuron.w[0] != 0 and neuron.w[1] != 0:
            vector_slope = neuron.w[1]/neuron.w[0]
            bound_slope = -1/vector_slope
            x = np.linspace(min_x, max_x, num=1000)
            y = bound_slope * x

            #Plot the line
            plt.plot(x, y)
        elif neuron.w[0] != 0 and neuron.w[1] == 0:
            #Plot the vertical line
            plt.axvline(x=0)
        elif neuron.w[0] == 0 and neuron.w[1] != 0:
            #Plot the horizontal line
            plt.axhline(y=0)

        #Show the graph
        plt.show()

### 3.2 Create Simplified Perceptron Class

We will set bias to 0 for simplicity.

In [ ]:
class Perceptron():
    def __init__(self,w):
        self.w = w

    def forward(self, x):
        activation = torch.dot(self.w,x)
        return activation

    def predict(self, x):
        with torch.no_grad():
            
            ################################
			# Fill Out Prediction Function #
			################################

### 3.3 Make Data For the Perceptron to Learn

In [ ]:
#Data
data=torch.tensor([(-1, -1), (-1, -0.5), (-1, 0), (-1, 0.5), (1, 0.5), (1, -0.5), (1, 1), (1, -1)],dtype=float)
label=torch.tensor([1,1,1,1,0,0,0,0], dtype=float)

min_x = min(data[:, 0])
max_x = max(data[:, 0])

data_size=len(data)
data_sum=torch.sum(data,dim=0)
data_mean=data_sum/data_size

x_coords_0 = [point[0] for point, label in zip(data, label) if label == 0]
y_coords_0 = [point[1] for point, label in zip(data, label) if label == 0]
x_coords_1 = [point[0] for point, label in zip(data, label) if label == 1]
y_coords_1 = [point[1] for point, label in zip(data, label) if label == 1]

### 3.4 Perceptron Learning with Gradient Descent

In [ ]:
# Define a learning rate
lr = 0.1
# Define a loss function
loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
#Finding Weight
weight = torch.tensor([random.random(), random.random()], requires_grad=True, dtype=float)
learning_rate = 0.5
neuron = Perceptron(weight)

# Initial Plot
if neuron != None:
    print('Initial Plot')
    with torch.no_grad():
        print(f"{neuron.w.tolist()} - {learning_rate}*{neuron.w.grad}")
        plot_perceptron(neuron, min_x, max_x, x_coords_0, y_coords_0, x_coords_1, y_coords_1)

# Iterate Through Epochs
for epoch in range(5):
    old_w = neuron.w
    all_correct = True
    
	# Iterate Through Data
    for i in range(len(data)):
        point = data[i]
        y = neuron.forward(point)

        # neuron update!
        loss_val = loss_fn(y, label[i])

        # Gradient calculation + weight update
        loss_val.backward()

        with torch.no_grad():
            # store old w
            old_w = neuron.w.detach().numpy().copy()

            # update w
            neuron.w -= lr * neuron.w.grad

            # Only Print Equation when prediction is wrong
            if label[i] != y_hat:
                print(f"{neuron.w.tolist()} - {learning_rate}*{neuron.w.grad}")

        # Only Make a New Plot when the weights change
        if label[i] != y_hat:
            with torch.no_grad():
                plot_perceptron(neuron, max_x, min_x, x_coords_0, y_coords_0, x_coords_1, y_coords_1)
                
        # Break loop if perceptron predicts everything correct
        with torch.no_grad():
            for i in range(len(data)):
                point = data[i]
                y_hat = neuron.predict(point)
                if y_hat != label[i]:
                    all_correct = False
            if all_correct:
                break
    if all_correct:
        break